In [1]:
# Path setup
import sys
import os

sys.path.append("/home/dchen/Random_Forest_Weights/")

# Basics:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Helpful:
from sklearn.model_selection import train_test_split

# Pipeline and ColumnsTransformer:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
# models:
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor

# my functions:
from src_rf.methods.calc_mean import *
from src_rf.methods.calc_weights import *
from src_rf.methods.calc_dist import *

### 1. Load Data

In [2]:
df = pd.read_csv("/home/dchen/Random_Forest_Weights/src_rf/data/energy_data_hourly.csv"
                 , index_col = 'datetime', parse_dates=True)

### 2. Train Test Split

In [3]:
X = df.drop('total_energy_usage', axis = 1)
y = df['total_energy_usage']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3 ,shuffle=False, random_state=42)

### 2. Load Weights

In [5]:
rf_energy_weights = np.load("/Data/Delong_BA_Data/rf_weights/energy_weights.npy")

In [6]:
rf_energy_weights.shape

(100, 75, 13636)

### 3. Random Forest

In [7]:
# 3.1 Parameters for Weight_Calculation:
bootstrap = True
max_samples = 0.5
# 3.2 Parameters for RF
n_estimators = 300
min_samples_split = 5
min_samples_leaf = 5
max_depth = 40.0

# 3.3 Model Training
rf = RandomForestRegressor(
    bootstrap=bootstrap, max_samples=max_samples,n_estimators = n_estimators, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, max_depth = max_depth, verbose=0, n_jobs=-1, random_state = 42
)
rf.fit(X_train, y_train)

RandomForestRegressor(max_samples=0.8, n_jobs=-1, random_state=42)

### 4. Sanity Check

In [8]:
rf_mean_weights = calc_mean_rf(rf_energy_weights, y_train)
rf_mean_normal = rf.predict(X_test)

In [9]:
# Are the two the same?
sum(np.round(rf_mean_weights, 5) == np.round(rf_mean_normal[0:rf_energy_weights.shape[1]], 5))

75